In [1]:
"""Sets the notebook to the project root"""

import sys, os
PROJECT_NAME = "analysis"
current_directory = os.getcwd().split("/")
while current_directory[-1] != PROJECT_NAME:
    current_directory = current_directory[:-1]
if len(current_directory) > 1:
    project_root = "/".join(current_directory)
    os.chdir(project_root)
else:
    raise Exception("ERROR: Project root not found")

from dotenv import load_dotenv
load_dotenv(dotenv_path="./env/env.env")

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
print("h")